In [1]:
import sys
import requests
import os
from pathlib import Path
from random import randint, random
from database import Persistence, CursorFromConnectionFromPool
from datetime import datetime
import pandas as pd
from psycopg2 import sql
import json
import utils
from bs4 import BeautifulSoup
from psycopg2.extras import RealDictCursor
import psycopg2


base_link = 'https://www.ourcommons.ca/members/en/'

# with CursorFromConnectionFromPool() as cur:
#     try:
#         legislator_data = pd.read_sql("SELECT * FROM ca_fed_legislators WHERE party='Liberal'", cur.connection)
#         # legislation_data = pd.read_sql("SELECT * FROM ca_fed_legislation WHERE pm_party='Liberal'", cur.connection)
#         cur.connection.commit()
#     except Exception as e:
#         print(f'An exception occured executing a query: \n{e}')

# params = list(legislator_data['source_id'])

db_host = 'openparl.cia2zobysfwo.us-west-2.rds.amazonaws.com'
db_port = '5432'
db_name = 'videoanalysis'
db_pass = 'Dedenne#3'
db_user = 's_leung'

with psycopg2.connect(host=db_host, password=db_pass, user=db_user, database=db_name, port='5432') as conn:
    with conn.cursor(cursor_factory=RealDictCursor) as cur:
        try:
            completed_dataset = pd.read_sql("SELECT * FROM completed_dataset", cur.connection)
            cur.connection.commit()
        except Exception as e:
            print(f'GG NO RE: {e}')

In [33]:
commons_link = 'https://www.ourcommons.ca/publicationsearch/en/?per=88867&pubType=37&View=D'

# for param in params:
#     print(requests.get(f'https://www.ourcommons.ca/publicationsearch/en/?per={param}&pubType=37&View=D'))

response = requests.get(commons_link)
soup = BeautifulSoup(response.content, 'lxml')
div_lst = soup.find('div', {'id':'Publications'}).find_all('div', {'class':'Publication PubType37'})


In [38]:
counter = {}

for item in div_lst:
    topics = [x.text for x in item.find('div', {'class':'pi-indexes'}).find_all('a')]
    for topic in topics:
        if topic in counter:
            counter[topic] += 1
        else:
            counter[topic] = 1

In [2]:
completed_dataset_dict = completed_dataset.to_dict('records')
len(completed_dataset_dict)

380804

In [3]:
persons = list(pd.DataFrame(completed_dataset_dict)['who_en'])
counter = {}
for person in persons:
    if type(person)==str:
        if '(' in person:
            try:
                party = person.split('(')[0]
                if party in counter:
                    counter[party]+=1
                else:
                    counter[party]=1
            except IndexError:
                pass 

names = [key.split('.')[1].strip() for key, value in counter.items() if '.' in key]

In [5]:
with CursorFromConnectionFromPool() as cur:
    try:
        legislator_data = pd.read_sql("SELECT * FROM ca_fed_legislators", cur.connection)
        # legislation_data = pd.read_sql("SELECT * FROM ca_fed_legislation WHERE pm_party='Liberal'", cur.connection)
        cur.connection.commit()
    except Exception as e:
        print(f'An exception occured executing a query: \n{e}')

params = list((legislator_data['name_full'], legislator_data['goverlytics_id']))

In [8]:
# make a dict


# legislators = []
# lipad_speakers = set(names)
# fed_legislators_set = set(params)
# # for item in lipad_speakers:
# #     if item in fed_legislators_set:
# #         legislators.append(item)
# print(lipad_speakers)
# print(fed_legislators_set)
